# Ensuring Consistency with ACID Transactions with Delta Lake (Loan Risk Data)

<img src="https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-logo-whitebackground.png" width=200/>

This is a companion notebook to provide a Delta Lake example against the Lending Club data.
* This notebook has been tested with *DBR 5.4 ML Beta, Python 3*

## The Data

The data used is public data from Lending Club. It includes all funded loans from 2012 to 2017. Each loan includes applicant information provided by the applicant as well as the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. For a full view of the data please view the data dictionary available [here](https://resources.lendingclub.com/LCDataDictionary.xlsx).


![Loan_Data](https://preview.ibb.co/d3tQ4R/Screen_Shot_2018_02_02_at_11_21_51_PM.png)

https://www.kaggle.com/wendykan/lending-club-loan-data

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Delta Lake

Optimization Layer a top blob storage for Reliability (i.e. ACID compliance) and Low Latency of Streaming + Batch data pipelines.

## Import Data and create pre-Delta Lake Table
* This will create a lot of small Parquet files emulating the typical small file problem that occurs with streaming or highly transactional data

In [5]:
# -----------------------------------------------
# Uncomment and run if this folder does not exist
# -----------------------------------------------
# Configure location of loanstats_2012_2017.parquet
lspq_path = "/databricks-datasets/samples/lending_club/parquet/"

# Read loanstats_2012_2017.parquet
data = spark.read.parquet(lspq_path)

# Reduce the amount of data (to run on DBCE)
(loan_stats, loan_stats_rest) = data.randomSplit([0.01, 0.99], seed=123)

# Select only the columns needed
loan_stats = loan_stats.select("addr_state", "loan_status")

# Create loan by state
loan_by_state = loan_stats.groupBy("addr_state").count()

# Create table
loan_by_state.createOrReplaceTempView("loan_by_state")

# Display loans by state
display(loan_by_state)

addr_state,count
SC,172
AZ,336
LA,171
MN,261
NJ,540
DC,40
OR,182
VA,424
RI,65
WY,31


## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Easily Convert Parquet to Delta Lake format
With Delta Lake, you can easily transform your Parquet data into Delta Lake format.

In [7]:
# Configure Delta Lake Silver Path
DELTALAKE_SILVER_PATH = "/ml/loan_by_state_delta"

# Remove folder if it exists
dbutils.fs.rm(DELTALAKE_SILVER_PATH, recurse=True)

Out[2]: False

In [8]:
%sql

describe extended loan_by_state_delta

col_name,data_type,comment
addr_state,string,null
count,bigint,null
amount,double,null
,,
# Detailed Table Information,,
Database,default,
Table,loan_by_state_delta,
Owner,root,
Created Time,Tue Nov 12 18:57:20 UTC 2019,
Last Access,Thu Jan 01 00:00:00 UTC 1970,


In [9]:
%sql 
-- Current example is creating a new table instead of in-place import so will need to change this code
DROP TABLE IF EXISTS loan_by_state_delta;

CREATE TABLE loan_by_state_delta
USING delta
LOCATION '/ml/loan_by_state_delta'
AS SELECT * FROM loan_by_state;

-- View Delta Lake table
SELECT * FROM loan_by_state_delta

addr_state,count
AR,104
WI,200
MS,88
WV,55
UT,77
OH,524
TX,1293
AL,190
IN,236
MA,315


In [10]:
%sql 
DESCRIBE DETAIL loan_by_state_delta

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,a13c3f92-cfc4-4787-af64-4dd70cdd1f5c,default.loan_by_state_delta,null,dbfs:/ml/loan_by_state_delta,2019-11-12T18:57:20.497+0000,2019-11-12T18:57:30.000+0000,List(),46,30506,Map(),1,2


## Stop the notebook before the streaming cell, in case of a "run all"

In [12]:
dbutils.notebook.exit("stop")

stop

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Unified Batch and Streaming Source and Sink

These cells showcase streaming and batch concurrent queries (inserts and reads)
* This notebook will run an `INSERT` every 10s against our `loan_stats_delta` table
* We will run two streaming queries concurrently against this data
* Note, you can also use `writeStream` but this version is easier to run in DBCE

In [14]:
# Read the insertion of data
loan_by_state_readStream = spark.readStream.format("delta").load(DELTALAKE_SILVER_PATH)
loan_by_state_readStream.createOrReplaceTempView("loan_by_state_readStream")

In [15]:
%sql
select addr_state, sum(`count`) as loans from loan_by_state_readStream group by addr_state

addr_state,loans
AZ,336
SC,172
LA,171
MN,261
NJ,540
DC,40
OR,182
VA,424
RI,65
WY,31


**Wait** until the stream is up and running before executing the code below

In [17]:
import time
i = 1
while i <= 6:
  # Execute Insert statement
  insert_sql = "INSERT INTO loan_by_state_delta VALUES ('IA', 450)"
  spark.sql(insert_sql)
  print('loan_by_state_delta: inserted new row of data, loop: [%s]' % i)
    
  # Loop through
  i = i + 1
  time.sleep(10)

loan_by_state_delta: inserted new row of data, loop: [1]
loan_by_state_delta: inserted new row of data, loop: [2]
loan_by_state_delta: inserted new row of data, loop: [3]
loan_by_state_delta: inserted new row of data, loop: [4]
loan_by_state_delta: inserted new row of data, loop: [5]
loan_by_state_delta: inserted new row of data, loop: [6]

**Note**: Once the previous cell is finished and the state of Iowa is fully populated in the map (in cell 14), click *Cancel* in Cell 14 to stop the `readStream`.

Let's review our current set of loans using our map visualization.

In [20]:
%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state_delta group by addr_state

addr_state,loans
AZ,336
SC,172
LA,171
MN,261
NJ,540
DC,40
OR,182
VA,424
RI,65
KY,164


Observe that the Iowa (middle state) has the largest number of loans due to the recent stream of data.  Note that the original `loan_by_state_delta` table is updated as we're reading `loan_by_state_readStream`.

##![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Full DML Support

**Note**: Full DML Support is a feature that will be coming soon to Delta Lake; the preview is currently available in Databricks.

Delta Lake supports standard DML including UPDATE, DELETE and MERGE INTO providing developers more controls to manage their big datasets.

Let's start by creating a traditional Parquet table

In [24]:
# Load new DataFrame based on current Delta table
lbs_df = sql("select * from loan_by_state_delta")

# Save DataFrame to Parquet
lbs_df.write.mode("overwrite").parquet("loan_by_state.parquet")

# Reload Parquet Data
lbs_pq = spark.read.parquet("loan_by_state.parquet")

# Create new table on this parquet data
lbs_pq.createOrReplaceTempView("loan_by_state_pq")

# Review data
display(sql("select * from loan_by_state_pq"))

addr_state,count
WA,336
ND,19
FL,1013
CO,297
KY,164
WY,31
CT,219
NE,41
MS,88
WV,55


###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) DELETE Support

The data was originally supposed to be assigned to `WA` state, so let's `DELETE` those values assigned to `IA`

In [26]:
%sql
-- Attempting to run `DELETE` on the Parquet table
DELETE FROM loan_by_state_pq WHERE addr_state = 'IA'

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: DELETE destination only supports Delta sources.
Some(Relation[addr_state#10553,count#10554L] parquet
);
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.notADeltaSourceException(DeltaErrors.scala:132)
	at com.databricks.sql.execution.PreprocessTableDelete$$anonfun$apply0$1.applyOrElse(PreprocessTableDelete.scala:51)
	at com.databricks.sql.execution.PreprocessTableDelete$$anonfun$apply0$1.applyOrElse(PreprocessTableDelete.scala:45)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1$$anonfun$2.apply(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1$$anonfun$2.apply(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:77)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1.apply(AnalysisHelper.scala:107)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1.apply(AnalysisHelper.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsDown(AnalysisHelper.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperators(AnalysisHelper.scala:73)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperators(LogicalPlan.scala:29)
	at com.databricks.sql.execution.PreprocessTableDelete.apply0(PreprocessTableDelete.scala:45)
	at com.databricks.sql.execution.PreprocessTableDelete.apply(PreprocessTableDelete.scala:41)
	at com.databricks.sql.execution.PreprocessTableDelete.apply(PreprocessTableDelete.scala:35)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:112)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:109)
	at scala.collection.LinearSeqOptimized$class.foldLeft(LinearSeqOptimized.scala:124)
	at scala.collection.immutable.List.foldLeft(List.scala:84)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:109)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:101)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.execute(RuleExecutor.scala:101)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.org$apache$spark$sql$catalyst$analysis$Analyzer$$executeSameContext(Analyzer.scala:136)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:130)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:102)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$executeAndTrack$1.apply(RuleExecutor.scala:80)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$executeAndTrack$1.apply(RuleExecutor.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:88)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeAndTrack(RuleExecutor.scala:79)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:114)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:113)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:201)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:113)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$analyzed$1.apply(QueryExecution.scala:82)
	at org.apache.spark.sql.execution.QueryExecution

**Note**: This command fails because the `DELETE` statements are not supported in Parquet, but are supported in Delta Lake.

In [28]:
%sql
-- Running `DELETE` on the Delta Lake table
DELETE FROM loan_by_state_delta WHERE addr_state = 'IA'

In [29]:
%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state_delta group by addr_state

addr_state,loans
AZ,336
SC,172
LA,171
MN,261
NJ,540
DC,40
OR,182
VA,424
RI,65
KY,164


###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) UPDATE Support
The data was originally supposed to be assigned to `WA` state, so let's `UPDATE` those values

In [31]:
%sql
-- Attempting to run `UPDATE` on the Parquet table
UPDATE loan_by_state_pq SET `count` = 2700 WHERE addr_state = 'WA'

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: UPDATE destination only supports Delta sources.
Some(Relation[addr_state#5294,count#5295L] parquet
);
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.notADeltaSourceException(DeltaErrors.scala:166)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge$$anonfun$apply0$1.applyOrElse(PreprocessTableUpdateEdge.scala:56)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge$$anonfun$apply0$1.applyOrElse(PreprocessTableUpdateEdge.scala:50)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1$$anonfun$2.apply(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1$$anonfun$2.apply(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:77)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1.apply(AnalysisHelper.scala:107)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1.apply(AnalysisHelper.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsDown(AnalysisHelper.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperators(AnalysisHelper.scala:73)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperators(LogicalPlan.scala:29)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge.apply0(PreprocessTableUpdateEdge.scala:50)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge.apply(PreprocessTableUpdateEdge.scala:46)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge.apply(PreprocessTableUpdateEdge.scala:39)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:112)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:109)
	at scala.collection.LinearSeqOptimized$class.foldLeft(LinearSeqOptimized.scala:124)
	at scala.collection.immutable.List.foldLeft(List.scala:84)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:109)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:101)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.execute(RuleExecutor.scala:101)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.org$apache$spark$sql$catalyst$analysis$Analyzer$$executeSameContext(Analyzer.scala:137)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:131)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:103)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$executeAndTrack$1.apply(RuleExecutor.scala:80)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$executeAndTrack$1.apply(RuleExecutor.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:88)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeAndTrack(RuleExecutor.scala:79)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:115)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:114)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:201)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:114)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$analyzed$1.ap

**Note**: This command fails because the `UPDATE` statements are not supported in Parquet, but are supported in Delta Lake.

In [33]:
%sql
-- Running `UPDATE` on the Delta Lake table
UPDATE loan_by_state_delta SET `count` = 2700 WHERE addr_state = 'WA'

In [34]:
%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state_delta group by addr_state

addr_state,loans
AZ,336
SC,172
LA,171
MN,261
NJ,540
DC,40
OR,182
VA,424
RI,65
KY,164


###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) MERGE INTO Support

#### INSERT or UPDATE parquet: 7-step process

With a legacy data pipeline, to insert or update a table, you must:
1. Identify the new rows to be inserted
2. Identify the rows that will be replaced (i.e. updated)
3. Identify all of the rows that are not impacted by the insert or update
4. Create a new temp based on all three insert statements
5. Delete the original table (and all of those associated files)
6. "Rename" the temp table back to the original table name
7. Drop the temp table

![](https://pages.databricks.com/rs/094-YMS-629/images/merge-into-legacy.gif)


#### INSERT or UPDATE with Delta Lake

2-step process: 
1. Identify rows to insert or update
2. Use `MERGE`

In [36]:
# Let's create a simple table to merge
items = [('IA', 0), ('CA', 2500), ('OR', 0)]
cols = ['addr_state', 'count']
merge_table = spark.createDataFrame(items, cols)
merge_table.createOrReplaceTempView("merge_table")
display(merge_table)

addr_state,count
IA,0
CA,2500
OR,0


Instead of writing separate `INSERT` and `UPDATE` statements, we can use a `MERGE` statement.

In [38]:
%sql
MERGE INTO loan_by_state_delta as d
USING merge_table as m
on d.addr_state = m.addr_state
WHEN MATCHED THEN 
  UPDATE SET *
WHEN NOT MATCHED 
  THEN INSERT *

In [39]:
%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state_delta group by addr_state

addr_state,loans
AZ,336
SC,172
LA,171
MN,261
NJ,540
DC,40
OR,0
VA,424
RI,65
KY,164


##![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Schema Evolution
With the `mergeSchema` option, you can evolve your Delta Lake table schema

In [41]:
# Generate new loans with dollar amounts 
loans = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as amount from loan_by_state_delta")
display(loans)

addr_state,count,amount
AR,43,430261.1530981482
WI,146,1462343.856379321
MS,17,174496.92816233894
WV,6,69927.99641217239
UT,9,92636.50749041872
IA,0,0.0
OH,232,2320948.9305149172
TX,1150,1.150612804027492E7
AL,51,518903.8830118387
IN,183,1837146.2695289524


In [42]:
# Let's write this data out to our Delta table
loans.write.format("delta").mode("append").save(DELTALAKE_SILVER_PATH)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o703.save.
: org.apache.spark.sql.AnalysisException: A schema mismatch detected when writing to the Delta table.
To enable schema migration, please set:
'.option("mergeSchema", "true")'.

Table schema:
root
-- addr_state: string (nullable = true)
-- count: long (nullable = true)


Data schema:
root
-- addr_state: string (nullable = true)
-- count: long (nullable = true)
-- amount: double (nullable = true)

 
If Table ACLs are enabled, these options will be ignored. Please use the ALTER TABLE
command for changing the schema.
 ;
	at com.databricks.sql.transaction.tahoe.MetadataMismatchErrorBuilder.finalizeAndThrow(DeltaErrors.scala:835)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation$class.updateMetadata(ImplicitMetadataOperation.scala:114)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.updateMetadata(WriteIntoDelta.scala:50)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.write(WriteIntoDelta.scala:90)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta$$anonfun$run$1$$anonfun$apply$1.apply(WriteIntoDelta.scala:71)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta$$anonfun$run$1$$anonfun$apply$1.apply(WriteIntoDelta.scala:70)
	at com.databricks.sql.transaction.tahoe.DeltaLog.withNewTransaction(DeltaLog.scala:408)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta$$anonfun$run$1.apply(WriteIntoDelta.scala:70)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta$$anonfun$run$1.apply(WriteIntoDelta.scala:69)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:812)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.run(WriteIntoDelta.scala:69)
	at com.databricks.sql.transaction.tahoe.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:128)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:45)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:86)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:146)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:187)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:183)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:113)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:113)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:111)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:240)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.sca

**Note**: This command fails because the schema of our new data does not match the schema of our original data

In [44]:
# Add the mergeSchema option
loans.write.option("mergeSchema","true").format("delta").mode("append").save(DELTALAKE_SILVER_PATH)

**Note**: With the `mergeSchema` option, we can merge these different schemas together.

In [46]:
%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`amount`) as amount from loan_by_state_delta group by addr_state order by sum(`amount`) desc limit 10

addr_state,amount
TX,1.150612804027492E7
NY,8864068.755074179
FL,6235322.962528119
CA,5245327.8966891365
PA,4264471.194009748
MA,2742999.6730522993
VA,2653245.278730863
GA,2596313.395921475
AZ,2468463.6847570487
WA,2377431.771556159


## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Let's Travel back in Time!
Databricks Delta’s time travel capabilities simplify building data pipelines for the following use cases. 

* Audit Data Changes
* Reproduce experiments & reports
* Rollbacks

As you write into a Delta table or directory, every operation is automatically versioned.

You can query by:
1. Using a timestamp
1. Using a version number

using Python, Scala, and/or Scala syntax; for these examples we will use the SQL syntax.  

For more information, refer to [Introducing Delta Time Travel for Large Scale Data Lakes](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html)

### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Review Delta Lake Table History
All the transactions for this table are stored within this table including the initial set of insertions, update, delete, merge, and inserts with schema modification

In [49]:
%sql
DESCRIBE HISTORY loan_by_state_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend
10,2019-11-12T19:12:25.000+0000,1950142778633211,bhavin.kukadia@databricks.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1775360236806696),0508-145148-fulls924,9,WriteSerializable,false
9,2019-11-12T19:11:41.000+0000,1950142778633211,bhavin.kukadia@databricks.com,MERGE,Map(predicate -> (d.`addr_state` = m.`addr_state`)),null,List(1775360236806696),0508-145148-fulls924,8,WriteSerializable,false
8,2019-11-12T19:05:44.000+0000,1950142778633211,bhavin.kukadia@databricks.com,UPDATE,Map(predicate -> (addr_state#5660 = WA)),null,List(1775360236806696),0508-145148-fulls924,7,WriteSerializable,false
7,2019-11-12T19:01:52.000+0000,1950142778633211,bhavin.kukadia@databricks.com,DELETE,"Map(predicate -> [""(default.loan_by_state_delta.`addr_state` = 'IA')""])",null,List(1775360236806696),0508-145148-fulls924,6,WriteSerializable,false
6,2019-11-12T19:01:08.000+0000,1950142778633211,bhavin.kukadia@databricks.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1775360236806696),0508-145148-fulls924,5,WriteSerializable,true
5,2019-11-12T19:00:55.000+0000,1950142778633211,bhavin.kukadia@databricks.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1775360236806696),0508-145148-fulls924,4,WriteSerializable,true
4,2019-11-12T19:00:42.000+0000,1950142778633211,bhavin.kukadia@databricks.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1775360236806696),0508-145148-fulls924,3,WriteSerializable,true
3,2019-11-12T19:00:29.000+0000,1950142778633211,bhavin.kukadia@databricks.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1775360236806696),0508-145148-fulls924,2,WriteSerializable,true
2,2019-11-12T19:00:17.000+0000,1950142778633211,bhavin.kukadia@databricks.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1775360236806696),0508-145148-fulls924,1,WriteSerializable,true
1,2019-11-12T19:00:05.000+0000,1950142778633211,bhavin.kukadia@databricks.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1775360236806696),0508-145148-fulls924,0,WriteSerializable,true


### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Time Travel via Version Number
Below are SQL syntax examples of Delta Time Travel by using a Version Number

In [51]:
%sql
SELECT * FROM loan_by_state_delta VERSION AS OF 0

addr_state,count
AR,104
WI,200
MS,88
WV,55
UT,77
OH,524
TX,1293
AL,190
IN,236
MA,315


In [52]:
%sql
SELECT * FROM loan_by_state_delta VERSION AS OF 9

addr_state,count
AR,104
WI,200
MS,88
WV,55
UT,77
IA,0
OH,524
TX,1293
AL,190
IN,236
